# Assignment 6
1. Use yeast dataset from UCI http://archive.ics.uci.edu/ml/machine-learning-databases/yeast/yeast.data
2. Remove the first column and use the last column as the target
3. Only leave CYT and VAC classes
4. Replace [0.3, 0.5, 0.7] in feature 2 to null
5. Replace [0.26, 0.36, 0.64] in feature 3 to null
6. Split the data

7. Impute the data (or not, it's your call)
8. Build a outlier detection model to classify VAC from CYT, i.e. 0 from 1
9. Build a classifer using sample augmentation techniques to flassify VAC from CYT, i.e. 0 from 1
10. Try different methods and hyper paramters

11. Report perfromance using F-1 score

In [123]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, chi2, RFE, SelectFromModel

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Import libraries
%matplotlib inline

plt.rcParams["figure.figsize"] = [16, 9]

In [124]:
# Load dataset
url = "https://people.arcada.fi/~martinel/"
names = ['Sequence Name','mcg', 'gvh', 'alm', 'mit', 'erl','pox','vac','nuc', 'label']
df = pd.read_csv(url + 'yeast.data', header=None, names=names, delim_whitespace=True)
df

,Sequence Name,mcg,gvh,alm,mit,erl,pox,vac,nuc,label
0,ADT1_YEAST,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,MIT
1,ADT2_YEAST,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,MIT
2,ADT3_YEAST,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,MIT
3,AAR2_YEAST,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,NUC
4,AATM_YEAST,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,MIT
...,...,...,...,...,...,...,...,...,...,...
1479,YUR1_YEAST,0.81,0.62,0.43,0.17,0.5,0.0,0.53,0.22,ME2
1480,ZIP1_YEAST,0.47,0.43,0.61,0.40,0.5,0.0,0.48,0.47,NUC
1481,ZNRP_YEAST,0.67,0.57,0.36,0.19,0.5,0.0,0.56,0.22,ME2
1482,ZUO1_YEAST,0.43,0.40,0.60,0.16,0.5,0.0,0.53,0.39,NUC


In [125]:
# Only leave CYT and VAC classes
df = (df.loc[(df['label'] == 'CYT') | (df['label'] == 'VAC')])
df.label.unique()

array(['CYT', 'VAC'], dtype=object)

In [126]:
# Remove the first column and use the last column as the target
y = df.iloc[:,-1]
df = df.iloc[: ,1:9]

In [127]:
# Replace [0.3, 0.5, 0.7] in feature 2 to null
df = df.replace({'gvh' : { 0.3:np.nan, 0.5:np.nan, 0.7:np.nan }})

# Replace [0.26, 0.36, 0.64] in feature 3 to null
df = df.replace({'alm' : { 0.26:np.nan, 0.36:np.nan, 0.64:np.nan }})

In [128]:
df['gvh'].isna().count()

493

In [129]:
df['erl'].isna().count()

493

In [130]:
df

,mcg,gvh,alm,mit,erl,pox,vac,nuc
5,0.51,0.40,0.56,0.17,0.5,0.5,0.49,0.22
9,0.40,0.39,0.60,0.15,0.5,0.0,0.58,0.30
12,0.40,0.42,0.57,0.35,0.5,0.0,0.53,0.25
15,0.46,0.44,0.52,0.11,0.5,0.0,0.50,0.22
16,0.47,0.39,0.50,0.11,0.5,0.0,0.49,0.40
...,...,...,...,...,...,...,...,...
1475,0.71,NaN,0.50,0.18,0.5,0.0,0.46,0.22
1476,0.61,0.48,0.54,0.25,0.5,0.0,0.50,0.22
1477,0.38,0.32,NaN,0.41,0.5,0.0,0.44,0.11
1478,0.38,0.40,0.66,0.35,0.5,0.0,0.43,0.11


In [131]:
y

5       CYT
9       CYT
12      CYT
15      CYT
16      CYT
       ... 
1475    CYT
1476    CYT
1477    CYT
1478    CYT
1483    CYT
Name: label, Length: 493, dtype: object